In [1]:
from collections import deque

## Part One

In [14]:
def parse(s):
    files = []
    free_blocks = deque()
    
    for id, file_size in enumerate(s[::2]):
        files.append((id, int(file_size)))
        
    for free_size in s[1::2]:
        free_blocks.append(int(free_size))
        
    return files, free_blocks

def compact(files, free_blocks):
    free_blocks = deque(free_blocks)
    files = deque(files)
    compact_id, compact_count = files.pop()
    pos = 0
    checksum = 0
    while files:
        try:
            file_id, size = files.popleft()
            for n in range(size):
                checksum += file_id * pos
                pos += 1
        except IndexError:
            break

        free = free_blocks.popleft()
        for n in range(free):
            checksum += compact_id * pos
            pos += 1
            compact_count -= 1
            if compact_count < 1:
                try:
                    compact_id, compact_count = files.pop()
                except IndexError:
                    break
    # straglers:
    for i in range(compact_count):
        checksum += pos * compact_id
        pos += 1        
    
    return checksum

In [15]:
s="2333133121414131402"
     
files, free_blocks = parse(s)
compact(files, free_blocks)

1928

In [4]:
with open('input_files/09.txt') as f:
    s = f.read()

files, free_blocks = parse(s)
print("part one: ", compact(files, free_blocks))

part one:  6323641412437


## Part Two

In [77]:
## Just for fun. Use a segment tree to allow finding open blocks quickly

class SegmentTree:
    def __init__(self, data, key):
        self.n = len(data)
        self.size = 1
        self.key=key

        while self.size < self.n:
            self.size *= 2

        print("Size", self.size)
        self.tree = [(-float('inf'), [])] * (2 * self.size)
        
        for i in range(self.n):
            self.tree[self.size + i] = data[i]
        
        for i in range(self.size - 1, 0, -1):
            self.tree[i] = max((self.tree[2*i], self.tree[2*i+1]), key=self.key)

    def __getitem__(self, idx):
        return self.tree[self.size + idx]
        
    def __iter__(self):
        for i in range(self.n):
            yield self.tree[self.size + i]
        
    def update(self, index, value):
        pos = self.size + index
        self.tree[pos] = value
        pos //= 2
        while pos > 0:
            self.tree[pos] = max((self.tree[2*pos], self.tree[2*pos+1]), key=self.key)
            pos //= 2
        
    def leftmost_fee_space(self, x):
        if self.key(self.tree[1]) < x:
            return -1
        idx = 1

        # While not at a leaf
        while idx < self.size:
            left_child = 2 * idx
            right_child = left_child + 1
            if self.key(self.tree[left_child]) >= x:
                idx = left_child
            else:
                idx = right_child
        
        return idx - self.size
        
def compact_whole_files(files, free_blocks):
    # move blocks in next slot with enough space
    # Keep track of remaining space and moved ids
    disk_blocks = SegmentTree([(space, []) for idx, space in enumerate(free_blocks)], key=lambda t: t[0])
    for back_index, (file_id, file_size) in enumerate(reversed(files)):
        # get the next block with available space
        # careful not to find space after this block!
        max_idx = len(free_blocks) - back_index
        found_idx = disk_blocks.leftmost_fee_space(file_size)
        if found_idx >= 0 and found_idx < max_idx:
            
            block_size, members = disk_blocks[found_idx]
            disk_blocks.update(found_idx, (block_size - file_size, members + [file_id] * file_size))
            files[-(back_index+1)] = (0, file_size)

    # calculate checksum
    pos = 0
    checksum = 0
    for original, [remaining, moved] in zip(files, disk_blocks):
        id, size = original
        for _ in range(size):
            checksum += pos * id
            pos += 1
        for id in moved:
            checksum += pos * id
            pos += 1
        pos += remaining
    return checksum

In [78]:
s="2333133121414131402"
  
files, free_blocks = parse(s)
compact_whole_files(files, free_blocks)

Size 16


2858

In [79]:
with open('input_files/09.txt') as f:
    s = f.read()

def parttwo():
    files, free_blocks = parse(s)
    return compact_whole_files(files, free_blocks)

print("Part Two:", parttwo())

Size 16384
Part Two: 6351801932670


In [21]:
%timeit parttwo()

56.2 ms ± 97.5 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
